In [3]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing

In [4]:
# 데이터 읽어오기- 1
train_file = './Chatbot_data/ChatbotData.csv'
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

In [6]:
# 단어 인덱스 시퀀스 벡터 - 2
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
#text_to_word_sequence를 이용해 단어 시퀀스를 만든 후 corpus에 저장
#다음 텐서플로 토크나이저의 texts_to_sequences를 이용해 문장 내 모든 단어를 시퀀스 번호로 변환한다.
#단어 시퀀스 벡터 크기를 맞추기 위해 밑에 있는 padded_seqs를 이용해 패딩 처리함

In [7]:
MAX_SEQ_LEN = 15 #단어 시퀀스 벡터 크기 - 3
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

#벡터 크기가 재각각이여서 학습시키려면 같게 만들어줘야함
# max_seq_len을 이용해 벡터 크기 맞춤
# 크기가 작은 단어는 공간이 남는데 그 부분을 0으로 채워준다.
# 이 과정을 패딩 padding이라 한다. 
# 케라스에서 pad_sequences() 함수를 통해 시퀀스의 패딩 처리를 손쉽게 할 수 있다.
# maxlen 인자로 시퀀스의 최대 길이를 정하는데 학습시킬 문장 데이터들을 사전에 분석해 파악해야한다.
# 너무 크면 자원 낭비, 작게 잡으면 데이터 손실이 발생

In [8]:
# 테스트용 데이터셋 생성 - 4
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
test_ds = ds.take(2000).batch(20) # 테스트 데이터셋

In [9]:
# CNN 모델 불러오기
model = load_model('cnn_model.h5')
model.summary()
model.evaluate(test_ds, verbose=2)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 15)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 15, 128)      1715072     ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 15, 128)      0           ['embedding[0][0]']              
                                                                                                  
 conv1d (Conv1D)                (None, 13, 128)      49280       ['dropout[0][0]']                
                                                                                              

[0.07340554147958755, 0.9769999980926514]

In [10]:
# 테스트용 데이터셋의 10212번째 데이터 출력 -5 
print('단어 시퀀스 : ', corpus[10212])
print('단어 인덱스 시퀀스 : ', padded_seqs[10212])
print('문장 분류(정답) : ', labels[10212])

#10212번째 문장의 단어 시퀀스와 시퀀스 번호 그리고 해당 문장의 감정 클래스 정답을 출력한 결과
#10212번째 문장은 사랑(label:2)으로 분류된 문장이다.
#실제 값


단어 시퀀스 :  ['썸', '타는', '여자가', '남사친', '만나러', '간다는데', '뭐라', '해']
단어 인덱스 시퀀스 :  [   13    61   127  4320  1333 12162   856    31     0     0     0     0
     0     0     0]
문장 분류(정답) :  2


In [11]:
# 감정 예측 - 6
picks = [10212]
predict = model.predict(padded_seqs[picks])
predict_class = tf.math.argmax(predict, axis=1)
print('감정 예측 점수 : ', predict)
print("감정 예측 클래스 : ", predict_class.numpy())
#argmax()를 이용해 분류 클래스들 중 예측 점수가 가장 큰 클래스 번호를 계산함
# 즉 10212번째 문장이 어떤 감정 클래스에 포함되어 있는지 판단함
#예측 값

1/1 [==============================] - 0s 128ms/step
감정 예측 점수 :  [[9.1608985e-07 1.8638556e-06 9.9999726e-01]]
감정 예측 클래스 :  [2]
